In [ ]:
# att1: 在retinanet fpn的基础上增加attent，输出还是3变5
# att2：在att1基础上，只输出3,4,5；相应的head也要改
# att3：在att2基础上，3,4,5输出max-pooling，维度和3一致，之后attn 3个输出；相应的head同2
# att4：在att3基础上，maxpooling改avg pooling
# att5: att2基础上输出n r loss，新head
# att6：att5基础上，不输出多余loss
# att7：att6基础上，只迭代一次

# att10：att2
# att11：att2+weather
# att12：att2+weather+wloss

In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from mmcv.cnn import ConvModule
from mmcv.runner import BaseModule, auto_fp16


In [3]:
a=torch.rand(16,256,80,80)
b=torch.rand(16,256,40,40)
c=torch.rand(16,256,20,20)
d=torch.rand(16,256,10,10)
e=torch.rand(16,256,5,5)

In [4]:
outs=[a,b,c,d,e]

In [ ]:
---------------------------------------------------------------

In [8]:
p_0_3, p_1_4, p_2_5, p_3_6, p_4_7  = outs

In [6]:
upsample_cfg=dict(mode='nearest')

In [7]:
        p_top = ConvModule(
                    256,
                    256,
                    3,
                    stride=2,
                    padding=1,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)

In [8]:
        p_3_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_4_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_5_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_6_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_7_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        level_weight_conv = ConvModule(
                    80,
                    5,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)

In [ ]:
-------------------------------------------------------------------

In [22]:
p_0_size = p_0_3.shape[2:]
p_1_size = p_1_4.shape[2:]
p_2_size = p_2_5.shape[2:]
p_3_size = p_3_6.shape[2:]
p_4_size = p_4_7.shape[2:]

In [23]:
p_0_3 = p_0_3
p_1_3 = p_top(p_0_3)
p_2_3 = p_top(p_1_3)
p_3_3 = p_top(p_2_3)
p_4_3 = p_top(p_3_3)

p_1_4 = p_1_4
p_2_4 = p_top(p_1_4)
p_3_4 = p_top(p_2_4)
p_4_4 = p_top(p_3_4)
p_0_4 = F.interpolate(p_1_4, size=p_0_size, **upsample_cfg)

p_2_5 = p_2_5
p_3_5 = p_top(p_2_5)
p_4_5 = p_top(p_3_5)
p_1_5 = F.interpolate(p_2_5, size=p_1_size, **upsample_cfg)
p_0_5 = F.interpolate(p_1_5, size=p_0_size, **upsample_cfg)

p_3_6 = p_3_6
p_4_6 = p_top(p_3_6)
p_2_6 = F.interpolate(p_3_6, size=p_2_size, **upsample_cfg)
p_1_6 = F.interpolate(p_2_6, size=p_1_size, **upsample_cfg)
p_0_6 = F.interpolate(p_1_6, size=p_0_size, **upsample_cfg)

p_4_7 = p_4_7
p_3_7 = F.interpolate(p_4_7, size=p_3_size, **upsample_cfg)
p_2_7 = F.interpolate(p_3_7, size=p_2_size, **upsample_cfg)
p_1_7 = F.interpolate(p_2_7, size=p_1_size, **upsample_cfg)
p_0_7 = F.interpolate(p_1_7, size=p_0_size, **upsample_cfg)

In [26]:
        level_0_weight_3 = p_3_conv(p_0_3)
        level_0_weight_4 = p_4_conv(p_0_4)
        level_0_weight_5 = p_5_conv(p_0_5)
        level_0_weight_6 = p_6_conv(p_0_6)
        level_0_weight_7 = p_7_conv(p_0_7)
        
        level_0_weight_concat = torch.cat((level_0_weight_3, level_0_weight_4, level_0_weight_5, level_0_weight_6, level_0_weight_7),1)
        level_0_weight = level_weight_conv(level_0_weight_concat)
        level_0_weight = F.softmax(level_0_weight, dim=1)

        level_0 = p_0_3 * level_0_weight[:,0:1,:,:]+\
                            p_0_4 * level_0_weight[:,1:2,:,:]+\
                            p_0_5 * level_0_weight[:,2:3,:,:]+\
                            p_0_6 * level_0_weight[:,3:4,:,:]+\
                            p_0_7 * level_0_weight[:,4:,:,:]




In [29]:
p_features=[[p_0_3,p_0_4,p_0_5,p_0_6,p_0_7],
[p_1_3,p_1_4,p_1_5,p_1_6,p_1_7],
[p_2_3,p_2_4,p_2_5,p_2_6,p_2_7],
[p_3_3,p_3_4,p_3_5,p_3_6,p_3_7],
[p_4_3,p_4_4,p_4_5,p_4_6,p_4_7],]

In [30]:
out_feature=[]
for i in range(5):
        level_weight_3 = p_3_conv(p_features[i][0])
        level_weight_4 = p_4_conv(p_features[i][1])
        level_weight_5 = p_5_conv(p_features[i][2])
        level_weight_6 = p_6_conv(p_features[i][3])
        level_weight_7 = p_7_conv(p_features[i][4])
        
        level_weight_concat = torch.cat((level_weight_3, level_weight_4, level_weight_5, level_weight_6, level_weight_7),1)
        level_weight = level_weight_conv(level_weight_concat)
        level_weight = F.softmax(level_weight, dim=1)

        level = p_features[i][0] * level_weight[:,0:1,:,:]+\
                  p_features[i][1] * level_weight[:,1:2,:,:]+\
                  p_features[i][2] * level_weight[:,2:3,:,:]+\
                  p_features[i][3] * level_weight[:,3:4,:,:]+\
                  p_features[i][4] * level_weight[:,4:,:,:]
        out_feature.append(level)



In [ ]:
        level_0_weight_v = self.weight_level_0(level_0_resized)
        level_1_weight_v = self.weight_level_1(level_1_resized)
        level_2_weight_v = self.weight_level_2(level_2_resized)
        levels_weight_v = torch.cat((level_0_weight_v, level_1_weight_v, level_2_weight_v),1)
        levels_weight = self.weight_levels(levels_weight_v)
        levels_weight = F.softmax(levels_weight, dim=1)

        fused_out_reduced = level_0_resized * levels_weight[:,0:1,:,:]+\
                            level_1_resized * levels_weight[:,1:2,:,:]+\
                            level_2_resized * levels_weight[:,2:,:,:]

        out = self.expand(fused_out_reduced)

In [ ]:
------------------------------------------------------------------------------

In [9]:
        p_top = ConvModule(
                    256,
                    256,
                    3,
                    stride=2,
                    padding=1,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_3_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_4_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        p_5_conv = ConvModule(
                    256,
                    16,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        # self.p_6_conv = ConvModule(
        #             256,
        #             16,
        #             1,
        #             stride=1,
        #             padding=0,
        #             conv_cfg=None,
        #             norm_cfg=None,
        #             act_cfg=None,
        #             inplace=False)
        # self.p_7_conv = ConvModule(
        #             256,
        #             16,
        #             1,
        #             stride=1,
        #             padding=0,
        #             conv_cfg=None,
        #             norm_cfg=None,
        #             act_cfg=None,
        #             inplace=False)
        level_weight_conv = ConvModule(
                    48,
                    3,
                    1,
                    stride=1,
                    padding=0,
                    conv_cfg=None,
                    norm_cfg=None,
                    act_cfg=None,
                    inplace=False)
        

In [11]:
        # outs=[p_0_3,          p_1_4,      p_2_5,      p_3_6,      p_4_7]
        #       60*60*256(8)    40*40*256   20*20*256   10*10*256   5*5*256
        p_0_3, p_1_4, p_2_5, p_3_6, p_4_7  = outs
        p_0_size = p_0_3.shape[2:]
        p_1_size = p_1_4.shape[2:]
        p_2_size = p_2_5.shape[2:]
        # p_3_size = p_3_6.shape[2:]
        # p_4_size = p_4_7.shape[2:]

        p_0_3 = p_0_3
        p_1_3 = p_top(p_0_3)
        p_2_3 = p_top(p_1_3)
        # p_3_3 = self.p_top(p_2_3)
        # p_4_3 = self.p_top(p_3_3)

        p_1_4 = p_1_4
        p_2_4 = p_top(p_1_4)
        # p_3_4 = self.p_top(p_2_4)
        # p_4_4 = self.p_top(p_3_4)
        p_0_4 = F.interpolate(p_1_4, size=p_0_size, **upsample_cfg)

        p_2_5 = p_2_5
        # p_3_5 = self.p_top(p_2_5)
        # p_4_5 = self.p_top(p_3_5)
        p_1_5 = F.interpolate(p_2_5, size=p_1_size, **upsample_cfg)
        p_0_5 = F.interpolate(p_1_5, size=p_0_size, **upsample_cfg)

        # p_3_6 = p_3_6
        # p_4_6 = self.p_top(p_3_6)
        # p_2_6 = F.interpolate(p_3_6, size=p_2_size, **self.upsample_cfg)
        # p_1_6 = F.interpolate(p_2_6, size=p_1_size, **self.upsample_cfg)
        # p_0_6 = F.interpolate(p_1_6, size=p_0_size, **self.upsample_cfg)

        # p_4_7 = p_4_7
        # p_3_7 = F.interpolate(p_4_7, size=p_3_size, **self.upsample_cfg)
        # p_2_7 = F.interpolate(p_3_7, size=p_2_size, **self.upsample_cfg)
        # p_1_7 = F.interpolate(p_2_7, size=p_1_size, **self.upsample_cfg)
        # p_0_7 = F.interpolate(p_1_7, size=p_0_size, **self.upsample_cfg)

        # p_features=[[p_0_3,p_0_4,p_0_5,p_0_6,p_0_7],
        #             [p_1_3,p_1_4,p_1_5,p_1_6,p_1_7],
        #             [p_2_3,p_2_4,p_2_5,p_2_6,p_2_7],
        #             [p_3_3,p_3_4,p_3_5,p_3_6,p_3_7],
        #             [p_4_3,p_4_4,p_4_5,p_4_6,p_4_7],]
        p_features=[[p_0_3,p_0_4,p_0_5],
            [p_1_3,p_1_4,p_1_5],
            [p_2_3,p_2_4,p_2_5]]

        out_feature=[]
        for i in range(3):
                level_weight_3 = p_3_conv(p_features[i][0])
                level_weight_4 = p_4_conv(p_features[i][1])
                level_weight_5 = p_5_conv(p_features[i][2])

                
                level_weight_concat = torch.cat((level_weight_3, level_weight_4, level_weight_5),1)
                level_weight = level_weight_conv(level_weight_concat)
                level_weight = F.softmax(level_weight, dim=1)

                level = p_features[i][0] * level_weight[:,0:1,:,:]+\
                        p_features[i][1] * level_weight[:,1:2,:,:]+\
                        p_features[i][2] * level_weight[:,2:,:,:]
                out_feature.append(level)

In [13]:
out_feature[0].shape

torch.Size([16, 256, 80, 80])

In [4]:
input = torch.randn(1, 256, 12, 12)

In [11]:
deconv2 = nn.ConvTranspose2d(256, 256, 3, stride=2, padding=1, output_padding =1)
deconv4 = nn.ConvTranspose2d(256, 256, 3, stride=4, padding=0, output_padding =1)

In [ ]:
# p_0_3 p_1_4 p_2_5

In [9]:
p_pool_3 = p_0_3

In [12]:
p_pool_4 = deconv2(p_1_4)
p_pool_5 = deconv4(p_2_5)

In [15]:
max = (p_pool_3+p_pool_4+p_pool_5)/3




In [16]:
max.shape

torch.Size([16, 256, 80, 80])

In [ ]:
---------------------------------------------------------------------------

In [5]:
p_0, p_1, p_2, p_3, p_4  = outs

In [7]:
import torch
import torch.nn as nn
import math
import numpy as np
from torchvision import transforms

In [9]:
		relu = nn.ReLU(inplace=True) 

		number_f = 32
		e_conv1 = nn.Conv2d(256,number_f,3,1,1,bias=True)
		e_conv2 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
		e_conv3 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
		e_conv4 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
		e_conv5 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
		e_conv7 = nn.Conv2d(number_f*2,8,3,1,1,bias=True) 
		e_conv8 = nn.Conv2d(number_f*2,8,3,1,1,bias=True) 
		maxpool = nn.MaxPool2d(2, stride=2, return_indices=False, ceil_mode=False)
		upsample = nn.UpsamplingBilinear2d(scale_factor=2)

In [12]:
		x=p_0
		x1 = relu(e_conv1(x))
		x2 = relu(e_conv2(x1))
		x3 = relu(e_conv3(torch.cat([x1,x2],1)))
		x4 = relu(e_conv4(torch.cat([x2,x3],1)))
		x5 = relu(e_conv5(torch.cat([x3,x4],1)))
		x_r = torch.tanh(e_conv7(torch.cat([x4,x5],1)))
		x_n = torch.tanh(e_conv8(torch.cat([x4,x5],1)))
		r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(x_r, 1, dim=1)
		n1,n2,n3,n4,n5,n6,n7,n8 = torch.split(x_n, 1, dim=1)

		x = (x-n1) * (r1+1)
		x = (x-n2) * (r2+1)
		x = (x-n3) * (r3+1)
		enhance_image_1 = (x-n4) * (r4+1)
		x = (enhance_image_1-n5) * (r5+1)
		x = (x-n6) * (r6+1)
		x = (x-n7) * (r7+1)
		enhance_image = (x-n8) * (r8+1)
		
		r = torch.cat([(r1+1),(r2+1),(r3+1),(r4+1),(r5+1),(r6+1),(r7+1),(r8+1)],1)
		n = torch.cat([n1,n2,n3,n4,n5,n6,n7,n8],1)
		# return enhance_image,r, n

In [19]:
r.shape

torch.Size([16, 8, 80, 80])

In [26]:
enhance_image2= torch.cat([enhance_image,r,n],1)

In [27]:
enhance_image2.shape

torch.Size([16, 272, 80, 80])

In [28]:
        feat = enhance_image2[:,0:256,:,:]
        r = enhance_image2[:,256:264,:,:]
        n = enhance_image2[:,264:272,:,:]

In [31]:
enhance_image2.shape

torch.Size([16, 272, 80, 80])

In [33]:
enhance_image2[:,-8:,:,:].shape

torch.Size([16, 8, 80, 80])

In [35]:
enhance_image2[:,-16:-8,:,:].shape

torch.Size([16, 8, 80, 80])

In [36]:
enhance_image2[:,:-16,:,:].shape

torch.Size([16, 256, 80, 80])

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torchvision.models.vgg import vgg16
#import pytorch_colors as colors
import numpy as np

from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from torchvision import transforms
import cv2
class L_color(nn.Module):

    def __init__(self):
        super(L_color, self).__init__()

    def forward(self, x):
        #bias = 0.00001
        #x = torch.div(x_o,x_i+bias)
        #x = x_o
        b,c,h,w = x.shape
        '''
        mean_rgb = torch.mean(x,[2,3],keepdim=True)
        mr,mg, mb = torch.split(mean_rgb, 1, dim=1)
        Drg = torch.pow(mr-mg,2)
        Drb = torch.pow(mr-mb,2)
        Dgb = torch.pow(mb-mg,2)
        k = torch.pow(torch.pow(Drg,2) + torch.pow(Drb,2) + torch.pow(Dgb,2),0.5)
        '''
        r, g, b = torch.split(x, 1, dim=1)
        pow_r = torch.norm(r, dim=(2,3))
        pow_g = torch.norm(g, dim=(2,3))
        pow_b = torch.norm(b, dim=(2,3))

        return (torch.pow(pow_r - pow_g, 2) + torch.pow(pow_r - pow_b, 2) + torch.pow(pow_b - pow_g, 2)) / (h*w)


def weights_map(Size):
    V=np.zeros(Size)
    x = np.append(np.matrix(np.arange(0,8)).getA(),np.matrix(np.arange(7,-1,-1)).getA())
    for i in range(0,Size[0]):
        for j in range(0,Size[1]):
            V[i][j] = math.log(math.e + math.sqrt(x[i]**2+x[j]**2))
    return torch.from_numpy(V).cuda()
class L_cen(nn.Module):

    def __init__(self,patch_size,mean_val):
        super(L_cen, self).__init__()
        # print(1)
        self.pool = nn.AvgPool2d(patch_size)
        self.mean_val = mean_val

    def forward(self, x ):

        b,c,h,w = x.shape
        x = torch.mean(x,1,keepdim=True)
        mean = self.pool(x)

        d = torch.mean(torch.pow((mean- torch.FloatTensor([self.mean_val] ).cuda())* weights_map((16,16)).squeeze(0).squeeze(0) ,2))
        return d



class L_ill(nn.Module):
    def __init__(self):
        super(L_ill,self).__init__()

    def forward(self,x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h =  (x.size()[2]-1) * x.size()[3]
        count_w = x.size()[2] * (x.size()[3] - 1)
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()
        return (h_tv/count_h+w_tv/count_w)/batch_size


class perception_loss(nn.Module):
    def __init__(self):
        super(perception_loss, self).__init__()
        features = vgg16(pretrained=True).features.cuda()
        self.to_relu_1_2 = nn.Sequential() 
        self.to_relu_2_2 = nn.Sequential() 
        self.to_relu_3_3 = nn.Sequential()
        self.to_relu_4_3 = nn.Sequential()

        for x in range(4):
            self.to_relu_1_2.add_module(str(x), features[x])
        for x in range(4, 9):
            self.to_relu_2_2.add_module(str(x), features[x])
        for x in range(9, 16):
            self.to_relu_3_3.add_module(str(x), features[x])
        for x in range(16, 23):
            self.to_relu_4_3.add_module(str(x), features[x])
        
        # don't need the gradients, just want the features
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        #x = x.cpu()
        h = self.to_relu_1_2(x)
        h_relu_1_2 = h
        h = self.to_relu_2_2(h)
        h_relu_2_2 = h
        h = self.to_relu_3_3(h)
        h_relu_3_3 = h
        h = self.to_relu_4_3(h)
        h_relu_4_3 = h
        # out = (h_relu_1_2, h_relu_2_2, h_relu_3_3, h_relu_4_3)
        return h_relu_4_3


class noise_loss(nn.Module):
    def __init__(self):
        super(noise_loss, self).__init__()
    
    def forward(self, x):
        batch_size = x.size()[0]
        loss = torch.pow(torch.norm(x, dim=(2,3)), 2)
        return torch.mean(loss,1)

In [20]:
        # L_ill(r)
        x=r
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h =  (x.size()[2]-1) * x.size()[3]
        count_w = x.size()[2] * (x.size()[3] - 1)
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()
        # return (h_tv/count_h+w_tv/count_w)/batch_size

In [21]:
(h_tv/count_h+w_tv/count_w)/batch_size

tensor(0.0015, grad_fn=<DivBackward0>)

In [24]:
# torch.mean(L_noi(n))
x=n
batch_size = x.size()[0]
loss = torch.pow(torch.norm(x, dim=(2,3)), 2)
# return torch.mean(loss,1)

In [25]:
torch.mean(torch.mean(loss,1))

tensor(2.4550, grad_fn=<MeanBackward0>)

In [ ]:
			enhanced_image,A,N  = DarkLighter(img_lowlight)

			Loss_ill = 1600*L_ill(A)
			
			loss_col = 50*torch.mean(L_color(enhanced_image))

			loss_cen = 10*torch.mean(L_cen(enhanced_image))
			
			loss_perc = 0.001*torch.norm(L_perc(enhanced_image) - L_perc(img_lowlight))

			loss_noise = 50*torch.mean(L_noi(N))

			loss =    Loss_ill   +loss_cen +  loss_col + loss_perc+ loss_noise

In [37]:
a=torch.rand(2, 8, 68, 92)

In [38]:
        x=a
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h =  (x.size()[2]-1) * x.size()[3]
        count_w = x.size()[2] * (x.size()[3] - 1)
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()


In [40]:
(h_tv/count_h+w_tv/count_w)/batch_size

tensor(2.6559)